In [3]:
import os

In [4]:
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification\\notebooks'

In [5]:
os.chdir("../")
%pwd

'd:\\Deep_Learning_Projects\\Sports_Classification'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data_path : list
    epochs : int
    batch_size : int
    is_augmented : bool
    image_size : list

In [7]:
from src.Sports_Classification.Constants import *
from src.Sports_Classification.utils.utils import *
import tensorflow as tf

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config["artifacts_root"]])


    def get_model_training_config(self) -> ModelTrainingConfig:
        training = self.config['training']
        prepare_base_model = self.config['base_model']
        params = self.params
        training_data = os.path.join(self.config['data_ingestion']['unzip_dir'], "data/train")

        create_directories([Path(training['root_dir'])])

        training_config = ModelTrainingConfig(
            root_dir = Path(training['root_dir']),
            trained_model_path = Path(training['trained_model_path']),
            updated_base_model_path = Path(prepare_base_model['updated_base_model_path']),
            training_data_path = Path(training_data),
            epochs = params['EPOCHS'],
            batch_size = params['BATCH_SIZE'],
            is_augmented = params['AUGMENTATION'],
            image_size = params['IMAGE_SIZE']
        )


        return training_config

In [9]:
import os, zipfile
from src.Sports_Classification.logger import logging
from src.Sports_Classification.exception import CustomException
import urllib.request as request
from src.Sports_Classification.utils.utils import *

In [10]:
class Model_Training:

    def __init__(self, config : ModelTrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    
    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1/255,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size = self.config.image_size[:-1],
            batch_size = self.config.batch_size,
            interpolation = "bilinear"
        )


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )


        if self.config.is_augmented:
            train_datagenertor = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range = 0.2,
                **datagenerator_kwargs
            )
        
        else:
            train_datagenertor = valid_datagenerator

        self.train_generator = train_datagenertor.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "training",
            shuffle = True,
            **dataflow_kwargs
        )


    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.train_generator.samples // self.train_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs = self.config.epochs,
            steps_per_epoch = self.steps_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator,
        )

        save_model(
            path = self.config.trained_model_path,
            model = self.model
        )


In [11]:
try:
    config = ConfigurationManager()
    model_training_config = config.get_model_training_config()
    model_training = Model_Training(config=model_training_config)
    model_training.get_base_model()
    model_training.train_valid_generator()
    model_training.train()

except Exception as e:
    raise CustomException(e, sys)

Found 2057 images belonging to 80 classes.
Found 8370 images belonging to 80 classes.
523/523 [==============================] - 3873s 7s/step - loss: 13.6748 - accuracy: 0.0504 - val_loss: 9.3425 - val_accuracy: 0.1269


c:\Users\admin\anaconda3\envs\sports_classification\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
